## Import libraries

In [ ]:
import os

import copick
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

## Load data

In [ ]:
copick_filepath = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/copick.config"
numpy_anns_sphere_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/numpy_sphere"
numpy_anns_point_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/numpy_point"
experiments_dir = "/home/michele/DNN/kaggle/kaggle_cryo_et/dataset/train/static/ExperimentRuns"
voxel_spacing = 10
tomo_type = "denoised"
available_runs = os.listdir(experiments_dir)
print(f"Available runs: {available_runs}")
root_copick = copick.from_file(copick_filepath)

### Select data

In [ ]:
experiment_to_show = "TS_5_4"
annotation_type = "sphere"

if annotation_type == "sphere":
    experiment_ann = np.load(os.path.join(numpy_anns_sphere_dir, experiment_to_show + ".npy"))
if annotation_type == "point":
    experiment_ann = np.load(os.path.join(numpy_anns_point_dir, experiment_to_show + ".npy"))

## Visualize data

### Draw dense volumes from processed numpy

WARNING: Visualization as dense voxels is too slow, please skip this cell

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
for obj in root_copick.pickable_objects:
    if not obj.is_particle:
        continue
    print(f"Name: {obj.name}, radius: {obj.radius}, color: {obj.color}")
    # Set color corresponding to label
    particle_type_bool = experiment_ann == obj.label
    ax.voxels(filled=particle_type_bool, facecolors=list(obj.color[:3]))
ax.set_aspect('equal')
plt.show()

### Draw centroids from copick format

In [ ]:
run = next(iter([single_run for single_run in root_copick.runs if single_run.name == experiment_to_show]))
print(run)

ax = plt.figure().add_subplot(projection='3d')
xs = []
ys = []
zs = []
colors = []
for obj in root_copick.pickable_objects:
    if not obj.is_particle:
        continue
    print(f"Name: {obj.name}, radius: {obj.radius}, color: {obj.color}")
    picks = run.get_picks(obj.name, user_id="curation", session_id="0")
    assert len(picks) == 1
    # Visualize each annotation independently
    for point in picks[0].points:
        # Center
        xs.append(round(point.location.x / voxel_spacing))
        ys.append(round(point.location.y / voxel_spacing))
        zs.append(round(point.location.z / voxel_spacing))
        colors.append(np.array(obj.color[:3]) / 255)
ax.scatter(xs, ys, zs, c=colors)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()